# Extração IPVA MG

#### Instalação de bibliotecas

In [ ]:
!pip install -r requirements.txt

#### Download das tabelas

In [2]:
import requests
import os

anos = [2022, 2023, 2024, 2025]

for ano in anos:
    periodos_fabricacao = ['01a10anos', '11a20anos',  '21a30anos']
    for periodo in periodos_fabricacao:
        # Realizar download da tabela
        url = f'http://diarioeletronico.fazenda.mg.gov.br/opendiariogeral/export/sites/diarioeletronico/Publicacoes/SEFMG/IPVA/{ano}/ValordoIPVAveiculosde{periodo}.pdf'
        response = requests.get(url)
        
        # Definir destino e criar diretório
        file_name = f'bases/{ano}/ipva_{ano}_veiculos_{periodo}.pdf'
        os.makedirs(os.path.dirname(file_name), exist_ok=True)

        # Salvar arquivo
        with open(file_name, mode='wb') as file:
            file.write(response.content)
            print(f'Download {file_name} concluído') 

Download bases/2022/ipva_2022_veiculos_01a10anos.pdf concluído
Download bases/2022/ipva_2022_veiculos_11a20anos.pdf concluído
Download bases/2022/ipva_2022_veiculos_21a30anos.pdf concluído
Download bases/2023/ipva_2023_veiculos_01a10anos.pdf concluído
Download bases/2023/ipva_2023_veiculos_11a20anos.pdf concluído
Download bases/2023/ipva_2023_veiculos_21a30anos.pdf concluído
Download bases/2024/ipva_2024_veiculos_01a10anos.pdf concluído
Download bases/2024/ipva_2024_veiculos_11a20anos.pdf concluído
Download bases/2024/ipva_2024_veiculos_21a30anos.pdf concluído
Download bases/2025/ipva_2025_veiculos_01a10anos.pdf concluído
Download bases/2025/ipva_2025_veiculos_11a20anos.pdf concluído
Download bases/2025/ipva_2025_veiculos_21a30anos.pdf concluído


#### Extrair dados do PDF

In [3]:
from pypdf import PdfReader

import os 

import re
import numpy as np
import pandas as pd 

# Filtrar arquivos no diretório
find_files = []
for root, dirs, files in os.walk("bases"):   
   for name in files:
      if name.split('.')[1] == 'pdf':
        file = os.path.join(root, name)
        find_files.append(file)
        
# Interar entre os arquivos encontrados
files_read = []
for file in find_files:
    reader = PdfReader(file)
    print('Arquivo', file)

    ano_ipva = file.split('\\')[1]
    file_name = file.split(".")[0]

    df = pd.DataFrame()
    for i in range(len(reader.pages)):
        page = reader.pages[i]

        # Extrair texto mantendo o layout    
        text = page.extract_text(extraction_mode="layout")
        
        # Remover títulos e legendas 
        text = re.sub(
            r'\s*$|Legenda.*|(?<=AUTOMÓVEIS\/UTILITÁRIOS).*|(?<=CAMINHONETES\/PICAPES).*|(?<=CAMINHÕES\/CAV. MECÂNICOS).*|(?<=ÔNIBUS\/MICROÔNIBUS).*|(?<=MOTOCICLETAS\/SIMILARES).*|(?<=MOTOR CASA).*|Secretaria de Estado de Fazenda.*|Superintendência de.*|Em R\$ 1,00.*'
            ,''
            ,text)

        # Transformar resultado em linhas
        lines = [line for line in text.split('\n') if line.strip()]   

        # Dividir os valores em colunas
        values = []    
        for line in lines:
            values.append([column.strip() for column in line.split('   ') if column.strip()])

        # Ajustar a primeira linha movendo a categoria do veículo para o index 1 
        values.insert(1, values.pop(0)) 

        try:
            # Concatenar os dataframes
            if not df.shape[0]:
                df = pd.DataFrame(values[1:], columns=values[0])
            else:      
                df = pd.concat([df, pd.DataFrame(values[1:], columns=values[0])], ignore_index=True)
        except:
            print('Erro:', values)

    # Incluir ano de referência da tabela de IPVA
    df['AnoIPVA'] = ano_ipva

    # Incluir categoria do veículo em todos os registros
    df['CATEGORIA'] = df['MODELO/VERSÃO'].str.extract('(AUTOMÓVEIS/UTILITÁRIOS|CAMINHONETES/PICAPES|CAMINHÕES/CAV. MECÂNICOS|ÔNIBUS/MICROÔNIBUS|MOTOCICLETAS/SIMILARES|MOTOR CASA)', expand=False).ffill(axis=0)        

    # Incluir a fabricante em todos os registros
    df['FABRICANTE'] = df['MODELO/VERSÃO'].str.extract('FABRICANTE/MARCA: (.*)', expand=False).ffill(axis=0)

    # Apagar as linhas que possuem a coluna CodIPVA vazia e que o cabecalho se repete
    df.dropna(subset=["CodIPVA"], axis=0, inplace=True)
    df.drop(df[df['CodIPVA'] == 'CodIPVA'].index, axis=0, inplace=True)   

    # Resetar o index
    df.reset_index(drop=True, inplace=True)

    # Substituir caracter que aparece para anos sem valores
    df.replace('-', np.NaN, inplace=True)    
    
    print(len(reader.pages), 'páginas lidas', end='\n\n')

    # Unir dataframes em uma lista
    files_read.append(df)

Arquivo bases\2022\ipva_2022_veiculos_01a10anos.pdf
170 páginas lidas

Arquivo bases\2022\ipva_2022_veiculos_11a20anos.pdf
162 páginas lidas

Arquivo bases\2022\ipva_2022_veiculos_21a30anos.pdf
129 páginas lidas

Arquivo bases\2023\ipva_2023_veiculos_01a10anos.pdf
152 páginas lidas

Arquivo bases\2023\ipva_2023_veiculos_11a20anos.pdf
151 páginas lidas

Arquivo bases\2023\ipva_2023_veiculos_21a30anos.pdf
123 páginas lidas

Arquivo bases\2024\ipva_2024_veiculos_01a10anos.pdf
152 páginas lidas

Arquivo bases\2024\ipva_2024_veiculos_11a20anos.pdf
158 páginas lidas

Arquivo bases\2024\ipva_2024_veiculos_21a30anos.pdf
124 páginas lidas

Arquivo bases\2025\ipva_2025_veiculos_01a10anos.pdf
161 páginas lidas

Arquivo bases\2025\ipva_2025_veiculos_11a20anos.pdf
170 páginas lidas

Arquivo bases\2025\ipva_2025_veiculos_21a30anos.pdf
135 páginas lidas



#### Modelar a base

In [4]:
# Exibição do dataframe gerado ao concatenar todos os registros
df_completo = pd.concat(files_read, axis=0, ignore_index=True)
display(df_completo.head())

,MODELO/VERSÃO,CodIPVA,2021,2020,2019,2018,2017,2016,2015,2014,...,1998,1997,1996,1995,1994,1993,1992,2022,2023,2024
0,AGRALE/MARRUA 4WD,0204902DN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMERICAR/CLASSIC 427,0160900ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMERICAR/CLASSIC XK 120,0160901ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASA/CAUYPE SPORT,0161000ON,NaN,NaN,"815,19","791,19","715,98","655,20","596,79","538,38",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I/ASTON MARTIN RAPIDE CP,0105012ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"22.533,99",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Exportar modelos dos veículos

In [7]:
df_modelos = df_completo[['CodIPVA', 'CATEGORIA', 'FABRICANTE', 'MODELO/VERSÃO']].drop_duplicates(ignore_index=True).copy()

# Tratar duplicidades nos códigos por diferença entre demais campos
# as diferenças foram agrupados na mesma linha
df_modelos = df_modelos.groupby(['CodIPVA', 'CATEGORIA', 'FABRICANTE'])['MODELO/VERSÃO'].agg(', '.join).reset_index() # Mesmo código com descrição de modelos diferentes
df_modelos = df_modelos.groupby(['CodIPVA', 'CATEGORIA', 'MODELO/VERSÃO'])['FABRICANTE'].agg(', '.join).reset_index() # Mesmo código com fabricantes diferentes
df_modelos = df_modelos.groupby(['CodIPVA', 'FABRICANTE', 'MODELO/VERSÃO'])['CATEGORIA'].agg(', '.join).reset_index() # Mesmo código com categorias diferentes
df_modelos = df_modelos.groupby(['CodIPVA', 'CATEGORIA'])[['FABRICANTE', 'MODELO/VERSÃO']].agg(', '.join).reset_index() # Mesmo código com fabricante e modelo diferentes
df_modelos = df_modelos.groupby(['CodIPVA', 'FABRICANTE'])[['CATEGORIA', 'MODELO/VERSÃO']].agg(', '.join).reset_index() # Mesmo código com categoria e modelos diferentes

df_modelos.to_csv('modelos.csv', index=False)

display(df_modelos.head())
print('Quantidade de veículos', df_modelos.shape[0])

,CodIPVA,FABRICANTE,CATEGORIA,MODELO/VERSÃO
0,0000000DC,CHEVROLET,CAMINHONETES/PICAPES,GM/ARB STYLLEN
1,0000000DN,MERCEDES-BENZ,CAMINHÕES/CAV. MECÂNICOS,M.BENZ/1728 TRUKAM CA
2,0000000OC,FORD,CAMINHONETES/PICAPES,I/FORD F150 4X4SC RAPTOR
3,0000000ON,BMW,AUTOMÓVEIS/UTILITÁRIOS,I/BMW M3 COUPE
4,0000007ON,SHINERAY,MOTOCICLETAS/SIMILARES,SHINERAY/SE2


Quantidade de veículos 14829


##### Exportar taxas IPVA

In [6]:
# Definir colunas de ano para realizar unpivot
unpivot_columns = [column for column in list(df_completo.columns) if re.findall(r'[0-9].*', column)]

# Realizar a transformação das colunas em linhas
df_ipva = pd.melt(df_completo, id_vars=['CodIPVA', 'AnoIPVA'], value_vars=unpivot_columns, var_name='AnoVeículo', value_name='ValorIPVA')
df_ipva.dropna(subset=['ValorIPVA'], axis=0, inplace=True)
df_ipva.reset_index(drop=True, inplace=True)

# Convertar a coluna ValorIPVA para float
df_ipva['ValorIPVA'] = df_ipva['ValorIPVA'].str.replace('.', '', regex=False).str.replace(',', '.').astype(float)

df_ipva.to_csv('ipva.csv', index=False)

display(df_ipva.head())
print('Registros', df_ipva.shape[0])

,CodIPVA,AnoIPVA,AnoVeículo,ValorIPVA
0,0100117ON,2022,2021,7351.98
1,0100154ON,2022,2021,8387.19
2,0106193ON,2022,2021,7879.59
3,0106169ON,2022,2021,8929.98
4,0156909ON,2022,2021,8335.20


Registros 213586
